# Download Classification and Manually Edit

In [ ]:
import geopandas as gpd
import boto3
import os
import sys
sys.path.insert(0, './tools/')
from addImageCalc import addImageCalc

### User defines the validation site and the directory where files will be downloaded

In [ ]:
#Local directory where you want the classified image downloaded
downloadDir = '/Users/mbonnema/Documents/OPERA_calval/DSWx/FakeChip/review/'

In [ ]:
#Only specify one. Leave the other as ''. If more than one planet image for given chip, PLANET_ID must be specified
PLANET_ID = ''
SITE_NAME = '2_1'
assert((len(PLANET_ID) == 0) ^ (len(SITE_NAME) == 0))

### Open AWS session and read image and image_calc tables

In [ ]:
session = boto3.session.Session(profile_name='saml-pub')
s3 = session.resource('s3')
s3_client = session.client('s3')

In [ ]:
bucket_name = 'opera-calval-database-dswx'

In [ ]:
obj = s3.Object(bucket_name,'image.geojson')
image_table_data = obj.get()['Body']
imageTable = gpd.read_file(image_table_data)

In [ ]:
obj = s3.Object(bucket_name,'image_calc.geojson')
image_table_data = obj.get()['Body']
image_calcs = gpd.read_file(image_table_data)

In [ ]:
# This cell will show the number of planet images found for a given chip. If more than one, ensure the printed Planet
# ID matches the planet image used to generate the classification
temp = imageTable[['image_name', 'site_name']]
df_site2image = temp.set_index('site_name')
df_image2site = temp.set_index('image_name')
if not PLANET_ID:
    values = PLANET_ID = df_site2image.loc[SITE_NAME].tolist()
    PLANET_ID = values[0]
    print(f'There was {len(values)} planet images for this chip')
else:
    values = df_image2site.loc[PLANET_ID].tolist()
    SITE_NAME = values[0]
    print(f'There were {len(values)} chips for this planet_image')

(SITE_NAME, PLANET_ID)

In [ ]:
def downloadImage_calc(row,downloadDir):
    bucket = row.bucket.iloc[0]
    keys = row.s3_keys.iloc[0]
    keys = keys.split(',')
    for key in keys:
        filename = key.split('/')[-1]
        response = s3_client.download_file(bucket,key,downloadDir+filename)

### Search for specific image and classified image for the defined validation site

In [ ]:
#This cell shows all classifications of specified planet image
search = image_calcs[image_calcs.image_name == PLANET_ID]
search.head(20)

In [ ]:
#This cell selects the classificaiton with the highest 'version' value (i.e. the most recent version)
search = search[search.version==search['version'].max()]
imagecalc_row = search.iloc[[0]]
imagecalc_row.head()

### Download classified image to the specified directory

In [ ]:
downloadImage_calc(imagecalc_row,downloadDir)